* Parsing conversations



In [ ]:
!pip install telethon

In [ ]:
import configparser
import json
import asyncio
from datetime import date, datetime

from telethon.tl.patched import Message
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel

In [ ]:
# Reading Configs
config = configparser.ConfigParser()
config.read("config.ini")

In [ ]:
# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
phone = config['Telegram']['phone']
username = config['Telegram']['username']

In [ ]:
# Create the client and connect
client = TelegramClient(username, api_id, api_hash)

In [ ]:
# Scrapping telegram dialogs function
import re

def remove_emojis(text):
    # Unicode ranges for emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

conversations = []

async def main(phone):
    await client.start()
    print("Client Created")

    if not await client.is_user_authorized():
        await client.send_code_request(phone)
        try:
            await client.sign_in(phone, input('Enter the code: '))
        except SessionPasswordNeededError:
            await client.sign_in(password=input('Password: '))

    async with client:
        for dialog in await client.get_dialogs():
            if dialog.is_user and not dialog.entity.bot and not dialog.is_group and dialog.id != 766015334: # Ban saved messages and block chat with yourself
                print(f"Processing {dialog.name} with ID {dialog.id}")
                offset_id = 0
                limit = 100

                while True:
                    print(f"Current Offset ID is: {offset_id}")
                    history = await client(GetHistoryRequest(
                        peer=dialog.entity,
                        offset_id=offset_id,
                        offset_date=None,
                        add_offset=0,
                        limit=limit,
                        max_id=0,
                        min_id=0,
                        hash=0
                    ))

                    if not history.messages:
                        break

                    messages = history.messages

                    beginConversation = True
                    myMessagesCount = 0
                    peerCount = 0
                    conversation = []
                    for _message in messages[::-1]:
                        if isinstance(_message, Message) and len(_message.message) != 0:

                            if beginConversation:

                                if _message.out is False:
                                    text = remove_emojis(_message.message).strip()
                                    if len(text) > 0:
                                        beginConversation = False
                                        peerCount += 1
                                        conversation.append({'role': 'user', 'content': text})
                                    else:
                                        continue
                                else:
                                    continue

                            else:

                                if myMessagesCount < 2 or peerCount < 2:

                                    if _message.out is True:
                                        role = 'assistant'
                                        myMessagesCount += 1
                                    else:
                                        role = 'user'
                                        peerCount += 1

                                    text = remove_emojis(_message.message).strip()
                                    conversation.append({'role': role, 'content': text})

                                else:
                                    print(conversation)
                                    conversations.append(conversation)
                                    myMessagesCount = 0
                                    peerCount = 0
                                    beginConversation = True
                                    conversation = []
                    if len(conversation) != 0: conversations.append(conversation)
                    offset_id = messages[len(messages) - 1].id


In [ ]:
# Scrapping telegram dialogs (result in "conversations")
await main(phone)

In [ ]:
# Saving scraped messages
with open('conversationsNew.json', 'w') as outfile:
    json.dump(conversations, outfile)

* PEFT

In [7]:
!pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2 trl==0.7.4

In [ ]:
# !pip install -- quiet transformers -U  <-- May be required if you have transformers error

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
import torch
import torch.nn as nn
import transformers

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
# Downloading a model
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)

for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Tokenizer from vanilla Mistral-7B
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [ ]:
# Saving conversations
import json
import random

with open('YOUR PATH TO JSON CONVERSATIONS PATH', 'r') as file:
    conversations = json.load(file)

selected_conversations = random.sample(conversations, 2000)

In [ ]:
# Implement a dataset class (download data from json if needed)
class Dataset(torch.utils.data.Dataset):
    def __init__(self, conversations, tokenizer):
        self.conversations = conversations
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        conversation = self.conversations[index]
        bos = '<|im_start|>'
        eos = '<|im_end|>\n'

        finalPrompt = '<|im_start|>system\nТы студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.\n<|im_end|>\n'
        for message in conversation:
            if message['role'] == 'user':
                finalPrompt += bos + 'friend\n' + message['content'] + eos
            else:
                finalPrompt += bos + 'you\n' + message['content'] + eos


        tokenized_input = self.tokenizer(finalPrompt, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        return {key: val.squeeze(0) for key, val in tokenized_input.items()}

    def __len__(self):
        return len(self.conversations)


In [ ]:
# Creating dataset
train_data = Dataset(conversations, tokenizer)

In [ ]:
# Setting congifurations for training
peft_config = LoraConfig(
    lora_alpha=4,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules = ["q_proj", "k_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to=None,
    evaluation_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=None,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024
)

In [ ]:
# Training loop
trainer.train()

In [ ]:
# Saving model
trainer.model.save_pretrained('YOUR PATH TO SAVE THE MODEL')

Comparison section

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing saved model
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    load_in_4bit=True,
    torch_dtype=torch.float32  # weights are 4-bit; layernorms and activations are fp32
)

model_path = '/content/drive/MyDrive/NLP/NeuralTwinModelWeights'
peftModel = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    load_in_4bit=True,
    torch_dtype=torch.float32
)

peftModel.eval()
model.eval()

In [7]:
# Tokenizer from vanilla Mistral-7B
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [32]:
# Function that helps to decode an output from language model
def process(outputs):
  answers = []
  for output in outputs:
    index = output.find('<')
    if index == 0:
      pass
    elif index == -1:
      answers.append(output)
    else:
      answers.append(output[:index])
  return answers

# Function to generate response with prompt
def generate_response(prompt, model):
  encodedPrompt = tokenizer(prompt, return_tensors='pt', add_special_tokens=False)
  modelInputs = encodedPrompt.to(device)

  generated_ids = model.generate(**modelInputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)

  toTraverse = decoded[0][len(prompt):]
  answers = process(toTraverse.split('\n'))
  return answers

In [39]:
prompts =  [
    "<|im_start|>system\nТы студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.<|im_end|>\n<|im_start|>friend\nКак твои дела?<|im_end|>\n",
    "<|im_start|>system\nТы студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.<|im_end|>\n<|im_start|>friend\nКак тебе вчерашний концерт.<|im_end|>\n",
    "<|im_start|>system\nТы студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.<|im_end|>\n<|im_start|>friend\nПривет, че делаешь?<|im_end|>\n"
]

from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

for prompt in prompts:

    answers_before = generate_response(prompt, model)
    answers_after = generate_response(prompt, peftModel)
    rows.append(row_template.format(prompt, '\n'.join(answers_before), '\n'.join(answers_after)))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,AFTER
"`<|im_start|>system Ты студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.<|im_end|> <|im_start|>friend Как твои дела?<|im_end|> `","Все хорошо, спасибо. Я учился в университете, но теперь я работаю в компании. Какую компанию? Моя компания специализируется на разработке программ.",Нормально Я сдал Ахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахахаха
"`<|im_start|>system Ты студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.<|im_end|> <|im_start|>friend Как тебе вчерашний концерт.<|im_end|> `","Ты помнишь, что я смотрел концерт группы ""The Beatles"" в 1964 году. Ты помнишь, что я смотрел концерт группы ""The Beatles"" в 1964 году.",Не помню Но я помню что ты сказал что он был Ты сказал что он был Ты сказал что он был
"`<|im_start|>system Ты студент из Москвы, тебе 20 лет. Ты - моя копия, отвечай моим друзяьми на их сообщения в социальной сети.<|im_end|> <|im_start|>friend Привет, че делаешь?<|im_end|> `","Что делаешь, друг? Не помню, что ты делаешь. Я учился в университете. Что ты учился?",Да я вчера сдал А сегодня уже нет Я вчера сдал А сегодня уже нет
